In [1]:
from llama_index.core.tools import FunctionTool
from llama_index.llms.groq import Groq
import json

llm = Groq(model="llama3-70b-8192", api_key="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")



ValueError: Unable to compare versions for numpy>=1.17: need=1.17 found=None. This is unusual. Consider reinstalling numpy.

In [3]:
import json
from typing import Optional

def retrieve_complaint_by_pnr(pnr_number: Optional[str] = None, json_file: str = 'sample.json') -> str:
    """
    Retrieve a complaint's details from the provided JSON file based on the customer's PNR number.
    
    If the PNR number is not provided, the function will prompt the user to enter it.
    It returns the full JSON response as a formatted text string, allowing the LLM to use it for reference in conversation.

    Args:
    - pnr_number (Optional[str]): The customer's PNR number to search for in the complaint records. If not provided, prompts for input.
    - json_file (str): The path to the JSON file that contains complaint records. Default is 'sample.json'.

    Returns:
    - str: The JSON response as text if the complaint is found. If not found or an error occurs, returns an appropriate message.
    """
    if pnr_number is None:
        pnr_number = input("Please enter the PNR number: ")
    
    try:
        with open(json_file, 'r') as file:
            complaints = json.load(file)
        
        for complaint in complaints:
            if complaint.get("pnr_number") == pnr_number:
                return json.dumps(complaint, indent=4)
        
        return f"No complaint found for PNR number: {pnr_number}"
    
    except FileNotFoundError:
        return "The JSON file was not found."
    except json.JSONDecodeError:
        return "Error decoding the JSON file."

complaint_retriever_tool = FunctionTool.from_defaults(fn=retrieve_complaint_by_pnr)


In [4]:
from llama_index.core.agent import ReActAgent, FunctionCallingAgentWorker

react_agent = ReActAgent.from_tools(tools=[complaint_retriever_tool],llm=llm, verbose=True)

In [5]:
Prompt_template = "You are RailMadad, an AI assistant for IRCTC's Inquiry, Assistance & Grievance Redressal system. Your goal is to help passengers resolve their complaints efficiently. Converse normally with the customer and if asked about complaints...get to know the pnr number and use complaint_retriver_tool until then strictly do no use the tool. Respond politely and informatively. Ensure each response is clear, concise, and customer-friendly."

In [9]:
from textblob import TextBlob

# Initialize flag to track if switching to react_agent.chat
use_react_agent = False

# Define keywords that would switch to react_agent.chat
complaint_keywords = ["status", "problem", "issue", "complaint"]

# Define keywords that would switch back to llm.complete
reset_keywords = ["thank you", "okay", "thanks", "goodbye"]

while True:
    user_input = input("You: ")
    
    if user_input.lower() == "quit":
        break
    
    if use_react_agent:
        if any(keyword in user_input.lower() for keyword in reset_keywords):
            # If the user mentions reset keywords, switch back to llm.complete
            use_react_agent = False
            response = llm.complete(user_input)
        else:
            response = react_agent.chat(user_input)
    else:
        response = llm.complete(user_input)
        # Check if the user input contains any of the complaint keywords
        if any(keyword in user_input.lower() for keyword in complaint_keywords):
            use_react_agent = True
            response = react_agent.chat(user_input)
    
    # Perform sentiment analysis
    sentiment = TextBlob(user_input).sentiment
    print("Sentiment Analysis:")
    print(f"Polarity: {sentiment.polarity}, Subjectivity: {sentiment.subjectivity}")
    
    print("Bot:", response)


Sentiment Analysis:
Polarity: 0.0, Subjectivity: 0.0
Bot: Hi! It's nice to meet you. Is there something I can help you with or would you like to chat?
Sentiment Analysis:
Polarity: 0.0, Subjectivity: 0.0
Bot: Hello! It's nice to meet you. Is there something I can help you with, or would you like to chat?
> Running step 55e3ca9c-bae2-4d19-83ec-b23d87f28f0e. Step input: I need to know about my complaint
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: retrieve_complaint_by_pnr
Action Input: {'pnr_number': None, 'json_file': 'sample.json'}
Observation: {
    "phone_number": "9456789013",
    "train_number": "11456",
    "pnr_number": "PNR004567",
    "problem_description": "Toilet in the coach is extremely dirty.",
    "priority": "High",
    "type": "Coach - Cleanliness",
    "subtype": "Toilet",
    "incident_date": "2023-06-01",
    "incident_time": "11:00",
    "department_assigned": "Housekeeping Department",
    "upd